# Keras Model for Beginners + EDA + R&D

## 1. Imports

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

Matplotlib is building the font cache; this may take a moment.


## 2. Load Data

In [10]:
# 학습 데이터
train = pd.read_json('../data/train.json')
train.head()

,id,band_1,band_2,inc_angle,is_iceberg
0,dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
1,e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
2,58b2aaa0,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859,1
3,4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
4,271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0


In [11]:
# 테스트 데이터
test = pd.read_json('../data/test.json')
test.head()

,id,band_1,band_2,inc_angle
0,5941774d,"[-15.863251, -15.201077, -17.887735, -19.17248...","[-21.629612, -21.142353, -23.908337, -28.34524...",34.966400
1,4023181e,"[-26.058969497680664, -26.058969497680664, -26...","[-25.754207611083984, -25.754207611083984, -25...",32.615072
2,b20200e4,"[-14.14109992980957, -15.064241409301758, -17....","[-14.74563980102539, -14.590410232543945, -14....",37.505433
3,e7f018bb,"[-12.167478, -13.706167, -16.54837, -13.572674...","[-24.32222, -26.375538, -24.096739, -23.8769, ...",34.473900
4,4371c8c3,"[-23.37459373474121, -26.02718162536621, -28.1...","[-25.72234344482422, -27.011577606201172, -23....",43.918874


## 3. About the Data

Sentinel-1 SAT는 지구 위 약 680km 상공에 떠있다. 이 위성은 입사각에서 신호 펄스를 쏜 다음 다시 돌아올 때의 신호를 기록한다. 기본적으로 그러한 반사 신호를 **후방 산란(Backscatter) 계수**라고 한다. 우리가 받은 데이터는 다음과 같이 주어진 전통적인 형태의 백스캐터 계수이다.

$$σo(dB)=βo(dB)+10log10[sin(ip)/sin(ic)]$$

* $ip$ : 특정 픽셀의 입사각
* $ic$ : 이미지 중심에 대한 입사각
* $K$ : 상수

우리는 $σo$를 데이터에서 찾을 수 있다.

### 3.1. Now Coming to the Features of σo
기본적으로 $σo$는 신호가 산란되는 표면에 따라 달라진다. 예를 들어, 특정 입시각의 경우 다음과 같이 변화한다.

* WATER........... SETTLEMENTS........ AGRICULTURE........... BARREN........
* `HH` : -27.001 ................ 2.70252 ................. -12.7952 ................ -17.25790909
* `HV` :  -28.035 ................ -20.2665 .................. -21.4471 ................. -20.019

`HH`의 값은 다양하지만 `HV`는 그렇지 않은 것을 볼 수 있다. 나는 배에서의 산란을 위한 데이터는 없지만, 금속 물체는 얼음 물체와는 분명히 다를 것이다.

### 3.2. WTF is HH HV?
이 Sentinel 인공위성은 RISTSAT(인도 원격 감지 위성)에 해당하고 H 분극(polarization)에서는 핑(ping)만 전송하고, V 분극에서는 전송하지 않는다. 그 H-ping들은 흩어지고, 물체는 그들의 분극을 변화시키고 H와 V의 혼합으로 되돌아온다. Sentinel은 H-변환기만 가지고 있기 때문에, 회신 신호는 HH와 HV의 형태 뿐이다. Sentinel에는 V-ping 송신기가 없기 때문에 VV는 회신되지 않는다.

다시 돌아와 피처에 대해 설명하자면, 이 데모 코드의 목적을 달성하기 위해 두 밴드를 모두 추출하여 그 밴드의 평균을 세 번째 채널로 하여 3채널의 RGB와 동등한 것을 생성한다.

In [ ]:
# HH, HV, 평균으로 3개의 밴드 생성
X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train['band_1']])
X_band_2 = np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train['band_2']])
# 학습 데이터 생성
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis], ((X_band_1 + X_band_2)/2)[:, :, :, np.newaxis]], axis=1)